<div style="text-align:center; background-color:blue; padding:10px;">
  <h1 style="color:white;">Premier chargement en base de données rds</h1>
</div>


<div style="text-align:left; background-color:gray; padding:10px;">
  <h1 style="color:white;">Import des librairies et des fichiers annexes</h1>
</div>


In [1]:
# Librairies pour la décompression de fichiers et système
import gzip
from io import BytesIO
import sys

# Librairies pour le scrapping
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Visualisation de données
import pandas as pd

# Connection à RDS
from connection import db,cursor,s3,bucket_name,connection_with_sqlalchemy

# datas processing
import datas_processing

In [2]:
# ///////////////////////////////////////////////////////////////////////////////
# SUPPRESSION DES TABLES ET BASE DE DONNEES SI IL Y A UN ÉCHEC DANS LA PROCEDURE
# ///////////////////////////////////////////////////////////////////////////////

def delete_tables_and_db(name_db : str) -> None:
    """
    Fonction permettant de supprimer les tables et une base de données

    Args:
        name_db (str) : "my_data_base"

    Return:
        None
    """
    cursor.execute(f"USE {name_db}")
    cursor.execute("SET foreign_key_checks = 0")
    cursor.execute("SHOW TABLES;")
    # Parcourir la liste des tables et supprimer chacune d'entre elles
    for table_name in cursor.fetchall():
        drop_table_query = f"DROP TABLE {table_name[0]}"
        cursor.execute(drop_table_query)
        print(f"Table {table_name} supprimée avec succès.")

    # Valider et appliquer les modifications
    db.commit()

    cursor.execute(f"DROP DATABASE {name_db};")
    db.commit()
    return


# Ligne à décommenter pour recommencer la procédure :
delete_tables_and_db("datagouv")

Table ('COMMUNES',) supprimée avec succès.
Table ('DEPARTEMENTS',) supprimée avec succès.
Table ('REGIONS',) supprimée avec succès.
Table ('TYPES_BIENS',) supprimée avec succès.
Table ('VENTES',) supprimée avec succès.


<div style="text-align:left; background-color:gray; padding:10px;">
  <h1 style="color:white;">Création des tables dans la base de données</h1>
</div>


In [3]:
query="""
CREATE DATABASE IF NOT EXISTS datagouv;
"""
cursor.execute(query)
db.commit()

query="""
USE datagouv;
"""
cursor.execute(query)

# Lecture de fichier create_tables
with open('create_tables.sql') as file:
    sql_queries = file.read()
try :
    for query in sql_queries.split(';')[:-1]:
        query = query.strip()
        cursor.execute(query)
except Exception as e:
    print(f"Erreur dans la creation des tables : {e}")
    db.rollback()

query="""
SHOW TABLES;
"""
cursor.execute(query)
tables = cursor.fetchall()

# Afficher les tables
for table in tables:
    print(f"La table {table[0]} à bien été créée")

La table COMMUNES à bien été créée
La table DEPARTEMENTS à bien été créée
La table REGIONS à bien été créée
La table TYPES_BIENS à bien été créée
La table VENTES à bien été créée


<div style="text-align:left; background-color:gray; padding:10px;">
  <h1 style="color:white;">Récupération des datas sur :<br>
  L'api https://geo.api.gouv.fr/ <br>
  et Scrapping des datas sur :
  https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/</h1>
</div>


#### - Les régions depuis l'api

In [4]:
url_regions = "https://geo.api.gouv.fr/regions"
response_regions = requests.get(url_regions)
data_regions = response_regions.json()
df_regions = pd.DataFrame(data_regions)
df_regions = df_regions.loc[:,["code","nom"]]
df_regions = df_regions.drop_duplicates()
df_regions.columns = ['ID_REGION','Name_region']
df_regions.head()

,ID_REGION,Name_region
0,11,Île-de-France
1,24,Centre-Val de Loire
2,27,Bourgogne-Franche-Comté
3,28,Normandie
4,32,Hauts-de-France


In [5]:
# Insertion dans la table REGIONS
query="""
USE datagouv;
"""
cursor.execute(query)

df_regions['ID_REGION'] = df_regions.ID_REGION.astype(str)
df_regions['Name_region'] = df_regions.Name_region.astype(str)

for index, row in df_regions.iterrows():
    query="""
    INSERT IGNORE INTO REGIONS (ID_REGION,Name_region)
    VALUES (%s,%s)
    """
    cursor.execute(query,(row["ID_REGION"],row["Name_region"]))
db.commit()

query="""
SELECT * FROM REGIONS
"""
cursor.execute(query)
regions = cursor.fetchall()

# Afficher les tables
for region in regions:
    print(f"ID_region: {region[0]} -- Nom region : {region[1]}")

ID_region: 01 -- Nom region : Guadeloupe
ID_region: 02 -- Nom region : Martinique
ID_region: 03 -- Nom region : Guyane
ID_region: 04 -- Nom region : La Réunion
ID_region: 06 -- Nom region : Mayotte
ID_region: 11 -- Nom region : Île-de-France
ID_region: 24 -- Nom region : Centre-Val de Loire
ID_region: 27 -- Nom region : Bourgogne-Franche-Comté
ID_region: 28 -- Nom region : Normandie
ID_region: 32 -- Nom region : Hauts-de-France
ID_region: 44 -- Nom region : Grand Est
ID_region: 52 -- Nom region : Pays de la Loire
ID_region: 53 -- Nom region : Bretagne
ID_region: 75 -- Nom region : Nouvelle-Aquitaine
ID_region: 76 -- Nom region : Occitanie
ID_region: 84 -- Nom region : Auvergne-Rhône-Alpes
ID_region: 93 -- Nom region : Provence-Alpes-Côte d'Azur
ID_region: 94 -- Nom region : Corse


#### - Les départements depuis l'api

In [6]:
# Récupération des codes départements, noms départements et code régions
url = "https://geo.api.gouv.fr/departements"
response = requests.get(url)
data_dpts = response.json()
df_depts = pd.DataFrame(data_dpts)
df_depts = df_depts.loc[:,["code","nom","codeRegion"]]
df_depts = df_depts.drop_duplicates()
df_depts.columns = ['ID_DEPT', 'Name_departement', 'ID_REGION']
df_depts.head()

,ID_DEPT,Name_departement,ID_REGION
0,01,Ain,84
1,02,Aisne,32
2,03,Allier,84
3,04,Alpes-de-Haute-Provence,93
4,05,Hautes-Alpes,93


In [7]:
# Insertion dans la table DEPARTEMENTS
for index, row in df_depts.iterrows():
    query="""
    INSERT IGNORE INTO DEPARTEMENTS (ID_DEPT, Name_departement,	ID_REGION)
    VALUES (%s,%s,%s)
    """
    cursor.execute(query,(row["ID_DEPT"],row["Name_departement"],row["ID_REGION"]))
db.commit()

query="""
SELECT * FROM DEPARTEMENTS
LIMIT 10
"""
cursor.execute(query)
dapartements = cursor.fetchall()

# Afficher les tables
for departement in dapartements:
    print(f"Id departement: {departement[0]} -- Nom departement : {departement[1]} -- Id region : {departement[2]}")

Id departement: 01 -- Nom departement : Ain -- Id region : 84
Id departement: 02 -- Nom departement : Aisne -- Id region : 32
Id departement: 03 -- Nom departement : Allier -- Id region : 84
Id departement: 04 -- Nom departement : Alpes-de-Haute-Provence -- Id region : 93
Id departement: 05 -- Nom departement : Hautes-Alpes -- Id region : 93
Id departement: 06 -- Nom departement : Alpes-Maritimes -- Id region : 93
Id departement: 07 -- Nom departement : Ardèche -- Id region : 84
Id departement: 08 -- Nom departement : Ardennes -- Id region : 44
Id departement: 09 -- Nom departement : Ariège -- Id region : 76
Id departement: 10 -- Nom departement : Aube -- Id region : 44


#### - Les communes depuis l'api

In [8]:
# Récupération des codes communes , noms communes et code départements 
url = "https://geo.api.gouv.fr/communes"
response = requests.get(url)
data_communes = response.json()
df_communes = pd.DataFrame(data_communes)
df_communes = df_communes.loc[:,["code","nom","codeDepartement"]]
df_communes = df_communes.drop_duplicates()
df_communes.columns= ["ID_COMMUNE","NAME_COMMUNE","ID_DEPT"]
df_communes.head()

,ID_COMMUNE,NAME_COMMUNE,ID_DEPT
0,01001,L'Abergement-Clémenciat,01
1,01002,L'Abergement-de-Varey,01
2,01004,Ambérieu-en-Bugey,01
3,01005,Ambérieux-en-Dombes,01
4,01006,Ambléon,01


In [9]:
# Création d'une liste avec les ID_COMMUNE pour s'assurer de pouvoir joindre
# les tables COMMUNES et VENTES
liste_ID_COMMUNES = df_communes.ID_COMMUNE.to_list()

In [10]:
# Insertion dans la table COMMUNES
for index, row in df_communes.iterrows():
    query="""
    INSERT IGNORE INTO COMMUNES (ID_COMMUNE, NAME_COMMUNE,	ID_DEPT)
    VALUES (%s,%s,%s)
    """
    cursor.execute(query,(row["ID_COMMUNE"],row["NAME_COMMUNE"],row["ID_DEPT"]))
db.commit()

query="""
SELECT * FROM COMMUNES
LIMIT 10
"""
cursor.execute(query)
communes = cursor.fetchall()

# Afficher les tables
for commune in communes:
    print(f"Id commune: {commune[0]} -- Nom commune : {commune[1]} -- Id départment : {commune[2]}")

Id commune: 01001 -- Nom commune : L'Abergement-Clémenciat -- Id départment : 01
Id commune: 01002 -- Nom commune : L'Abergement-de-Varey -- Id départment : 01
Id commune: 01004 -- Nom commune : Ambérieu-en-Bugey -- Id départment : 01
Id commune: 01005 -- Nom commune : Ambérieux-en-Dombes -- Id départment : 01
Id commune: 01006 -- Nom commune : Ambléon -- Id départment : 01
Id commune: 01007 -- Nom commune : Ambronay -- Id départment : 01
Id commune: 01008 -- Nom commune : Ambutrix -- Id départment : 01
Id commune: 01009 -- Nom commune : Andert-et-Condon -- Id départment : 01
Id commune: 01010 -- Nom commune : Anglefort -- Id départment : 01
Id commune: 01011 -- Nom commune : Apremont -- Id départment : 01


In [11]:
# Insertion dans la table TYPES_BIENS
liste_type_local = ["Appartement","Maison"]
for local in liste_type_local:
    query="""
    INSERT IGNORE INTO TYPES_BIENS (NAME_TYPE_BIEN)
    VALUES(%s)
    """
    cursor.execute(query,(local))
db.commit()

query="""
SELECT * FROM TYPES_BIENS
"""
cursor.execute(query)
types_biens = cursor.fetchall()

# Affichage du contenu de la table TYPES_BIENS
for i in types_biens:
    print(f"Id type bien : {i[0]} -- nom : {i[1]}")

Id type bien : 1 -- nom : Appartement
Id type bien : 2 -- nom : Maison


#### - Les ventes en webscrapping depuis https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/

In [12]:
# URL de la page web
url = 'https://files.data.gouv.fr/geo-dvf/latest/csv/'

response = requests.get(url)
html_content = response.content

# Utilisez BeautifulSoup pour analyser le HTML
soup = BeautifulSoup(html_content, 'html.parser')

In [13]:
# Trouver toutes les balises <a>
a_tags = soup.find_all('a')
annees=[]
# Parcourir chaque balise <a> et extraire le texte ainsi que la date/heure
for a_tag in a_tags:
    name = a_tag.text.strip()
    if name!="../":
        annees.append(name)
print(annees)

['2018/', '2019/', '2020/', '2021/', '2022/', '2023/']


In [14]:
query="""
USE datagouv;
"""
cursor.execute(query)

# Utilisation de sqlalchemy pour charger les données datagouv dans RDS
# Création de la connection
engine = connection_with_sqlalchemy("datagouv")

# Commencez une transaction
with engine.begin() as transaction:
    try :
        # La dernière année ne sera pas prise en compte pour tester la mise à jour des 
        # données dans le programme maj_datas.py
        for annee in annees[:-1] : 
            url = f'https://files.data.gouv.fr/geo-dvf/latest/csv/{annee}'
            response = requests.get(url)
            html_content = response.content
            soup = BeautifulSoup(html_content, 'html.parser')

            # Trouver la balise 'a' qui contient les liens de téléchargement
            csv_element = soup.find('pre').find('a', {'href': "full.csv.gz"})
            csv_link = urljoin(url, csv_element['href'])

            # Téléchargement des datas et conversion en data frame
            csv_response = requests.get(csv_link)
            if csv_response.status_code == 200:
                # Utilisation du buffer pour décompresser le fichier .gz
                try :
                    with BytesIO(csv_response.content) as file_buffer:
                        with gzip.GzipFile(fileobj=file_buffer, mode='rb') as gz_file:
                            df = pd.read_csv(gz_file, low_memory=False)
                except Exception as e:
                    print("Erreur dans la transformation des données en data frame :",e)

            else :
                print("Problème dans le lien de récupération de données !!!")

            # Selection des données
            df = datas_processing.select_datas(df)
            
            # Gestion des valeurs manquantes :
            df = datas_processing.nan_management(df)

            # Adaptation des données du data frame au format de la base de données
            df = datas_processing.format_data(df)

            # Groupement des données sur une seule ligne par id_mutation (vente)
            df = datas_processing.grouped_datas(df)

            # Adaptation des données avec les variables et clés étrangères de la bdd
            df = datas_processing.features_and_foreign_keys(df,liste_ID_COMMUNES,annee)

            #Suppression des lignes dupliquées
            df = df.drop_duplicates()

            
            df.to_sql(name='VENTES', con=engine, if_exists='append', index=False)
            print(f"Lignes insérées pour {annee.split('/')[0]} : ",df.shape[0])

    except Exception as e:
        # transaction annulée automatiquement si aucune exception est levée
        print(f"Error: {e}")

Nombre de ligne à insérer pour l'année 2018 :  462066
Nombre de lignes ne pouvant être insérées : 28822
Ligne insérées pour 2018 :  433100
Nombre de ligne à insérer pour l'année 2019 :  952543
Nombre de lignes ne pouvant être insérées : 57809
Ligne insérées pour 2019 :  894523
Nombre de ligne à insérer pour l'année 2020 :  929474
Nombre de lignes ne pouvant être insérées : 50002
Ligne insérées pour 2020 :  879239
Nombre de ligne à insérer pour l'année 2021 :  1084560
Nombre de lignes ne pouvant être insérées : 58494
Ligne insérées pour 2021 :  1025835
Nombre de ligne à insérer pour l'année 2022 :  1033467
Nombre de lignes ne pouvant être insérées : 60815
Ligne insérées pour 2022 :  972374


In [15]:
query="""
SELECT 
    V.*,
    T.NAME_TYPE_BIEN,
    C.NAME_COMMUNE,
    D.Name_departement,
    R.Name_region
FROM VENTES V
INNER JOIN TYPES_BIENS as T ON V.ID_TYPE_BIEN = T.ID_TYPE_BIEN
INNER JOIN COMMUNES AS C ON V.ID_COMMUNE = C.ID_COMMUNE
INNER JOIN DEPARTEMENTS AS D ON C.ID_DEPT = D.ID_DEPT
INNER JOIN REGIONS R ON D.ID_REGION = R.ID_REGION
LIMIT 10;
"""
df = pd.read_sql(query, engine)
df.head(10)

,ID_VENTE,MONTANT,NUMERO_RUE,RUE,CODE_POSTAL,LONGITUDE,LATITUDE,DATE_MUTATION,SURFACE_BATI,NB_PIECES,SURFACE_TERRAIN,DEPENDANCES,ID_TYPE_BIEN,ID_COMMUNE,NAME_TYPE_BIEN,NAME_COMMUNE,Name_departement,Name_region
0,2,37000,5,RUE D YPRES,01000,5.220403,46.197326,2018-07-06,30,1,0,1,1,01053,Appartement,Bourg-en-Bresse,Ain,Auvergne-Rhône-Alpes
1,43,74000,5670,BD MORINET,16260,0.450180,45.825141,2018-08-02,32,2,621,1,1,16085,Appartement,Chasseneuil-sur-Bonnieure,Charente,Nouvelle-Aquitaine
2,48,50000,13,GR GRANDE RUE,16110,0.386824,45.740376,2018-08-01,113,5,0,1,1,16281,Appartement,La Rochefoucauld-en-Angoumois,Charente,Nouvelle-Aquitaine
3,87,45000,13,GR GRANDE RUE,16110,0.386824,45.740376,2018-08-09,103,3,0,1,1,16281,Appartement,La Rochefoucauld-en-Angoumois,Charente,Nouvelle-Aquitaine
4,118,120000,21,BD VICTOR HUGO,01000,5.229090,46.201418,2018-09-28,82,4,0,0,1,01053,Appartement,Bourg-en-Bresse,Ain,Auvergne-Rhône-Alpes
5,167,123000,2,RUE THEOPHILE GIBOUIN,16500,0.670092,46.015351,2018-08-31,48,3,2262,0,1,16106,Appartement,Confolens,Charente,Nouvelle-Aquitaine
6,179,254000,3,CHAMPS DE LA MONTEE,16510,0.225537,45.993701,2018-09-17,100,4,61625,0,1,16400,Appartement,Verteuil-sur-Charente,Charente,Nouvelle-Aquitaine
7,218,16000,41,RUE GABRIEL PERI,02100,3.283414,49.844160,2018-11-09,39,2,0,0,1,02691,Appartement,Saint-Quentin,Aisne,Hauts-de-France
8,233,60000,154,BD GAMBETTA,02100,3.292034,49.852991,2018-10-25,51,3,0,0,1,02691,Appartement,Saint-Quentin,Aisne,Hauts-de-France
9,265,47000,26,RUE DU VIEUX PORT,02100,3.277169,49.841145,2018-10-26,48,2,0,0,1,02691,Appartement,Saint-Quentin,Aisne,Hauts-de-France


In [16]:
# Fermeture des connexions
engine.dispose()
cursor.close()
db.close()

<div style="text-align:left; background-color:gray; padding:10px;">
  <h3 style="color:white;">Récupération de la dernière date de mise à jour des données et stockage dans le bucket S3</h3>
</div>


In [17]:
date_element = soup.find('pre').contents[-1].strip()
date_maj = date_element.replace(" ","")
print(date_maj)

# Utilisation de BytesIO pour créer un buffer en mémoire
buffer = BytesIO()
buffer.write(date_maj.encode('utf-8'))  # Encode la chaîne en UTF-8 avant de l'écrire

# Écriture du texte dans le fichier sur S3
buffer.seek(0)  # Réinitialise la position de lecture/écriture
s3.upload_fileobj(buffer, bucket_name, 'app_immo/last_maj.txt')

# Assurez-vous de fermer le buffer BytesIO
buffer.close()

13-Oct-202319:50123388316


In [18]:
# Téléchargement du fichier depuis S3 vers un nouveau buffer
buffer_read = BytesIO()
s3.download_fileobj(bucket_name, 'app_immo/last_maj.txt', buffer_read)

# Lecture du contenu du buffer
buffer_read.seek(0)
last_maj_saved = buffer_read.read().decode('utf-8') 
print(last_maj_saved)

13-Oct-202319:50123388316


In [19]:
# ajout d'une valeur quelconque pour tester la lambda 
start_value="start_value"

# Utilisation de BytesIO pour créer un buffer en mémoire
buffer = BytesIO()
buffer.write(start_value.encode('utf-8'))  

# Écriture du texte dans le fichier sur S3
buffer.seek(0)  # Réinitialise la position de lecture/écriture
s3.upload_fileobj(buffer, bucket_name, 'app_immo/last_maj.txt')

# Fermeture du buffer
buffer.close()